In [1]:
%%capture
%run Data_Cleaning.ipynb

In [2]:
# #### Calculate the metrics RMSE and MAPE ####
# def calculate_rmse(y_true, y_pred):
#     """
#     Calculate the Root Mean Squared Error (RMSE)
#     """
#     rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
#     return rmse


# def calculate_mape(y_true, y_pred):
#     """
#     Calculate the Mean Absolute Percentage Error (MAPE) %
#     """
#     y_pred, y_true = np.array(y_pred), np.array(y_true)
#     mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
#     return mape

## Model config

In [7]:
random_seed = 42
training_size = 0.85
validation_size = 0.1

# Get unique month periods
month_periods = sorted(three_ff_cleaned_df['Date'].dt.to_period('M').unique())
# Split size calculation
total_periods = three_ff_cleaned_df['Date'].dt.to_period('M').nunique()
train_size = int(total_periods * training_size)
val_size = int(total_periods * validation_size)
test_size = total_periods - train_size - val_size

# print(month_periods)
print("train_size is :", train_size, "months")
print("val_size is :", val_size, "months")
print("test_size is :", test_size, "months")

# ElasticNet
l1_ratios = [0.1, 0.5, 0.9]

# SVR
svr_C_values = [0.1, 1, 10]
svr_epsilons = [0.01, 0.1]

# XGboost
xgb_params = [(50, 0.1), (100, 0.05), (200, 0.01)]  # (n_estimators, learning_rate))

# DecisionTreeRegressor
tree_depths = [2, 4, 6, 8, 10]

print("##################################")
print("####### Hyperparameters ##########")
print("For Ridge: alphas = [0.01, 0.1, 1.0, 10.0, 100.0]")
print("For Lasso: alphas = [0.01, 0.1, 1.0, 10.0, 100.0]")
print("For ElasticNet: l1_ratios =", l1_ratios)
print("For SVR - svr_C_values:", svr_C_values)
print("For SVR - svr_epsilons:", svr_epsilons)
print("For Xgboost - n_estimators and learning rate", xgb_params)
print("For DecisionTreeRegressor", tree_depths)

train_size is : 182 months
val_size is : 21 months
test_size is : 12 months
##################################
####### Hyperparameters ##########
For Ridge: alphas = [0.01, 0.1, 1.0, 10.0, 100.0]
For Lasso: alphas = [0.01, 0.1, 1.0, 10.0, 100.0]
For ElasticNet: l1_ratios = [0.1, 0.5, 0.9]
For SVR - svr_C_values: [0.1, 1, 10]
For SVR - svr_epsilons: [0.01, 0.1]
For Xgboost - n_estimators and learning rate [(50, 0.1), (100, 0.05), (200, 0.01)]
For DecisionTreeRegressor [2, 4, 6, 8, 10]


## 3 Factor Fama French Modelling

In [8]:
# Convert all columns except 'Date' to float
three_ff_cleaned_df = three_ff_cleaned_df.sort_values(by='Date').reset_index(drop=True)
three_ff_cleaned_df.head(4)

,Date,AAPL.O,NVDA.O,MSFT.O,AMZN.O,LLY,WMT,XOM,MA,UNH,...,ASTI.O,AWH.O,SUNE.O,TCRT.O,WINT.O,DGLY.O,Mkt-RF,SMB,HML,RF
0,2007-02-28,-1.315037,1.135388,-9.120327,3.828094,-2.867782,1.291682,-3.320376,-3.996326,-0.153198,...,36.080434,-27.996003,4.710145,-0.564442,-13.182539,-16.907633,-1.96,1.19,-0.14,0.38
1,2007-03-31,9.357873,-7.430645,-1.070674,1.647066,2.107322,-2.855538,5.125842,-0.880898,1.502644,...,66.263841,7.410797,-1.991531,-3.650742,7.443259,14.090545,0.68,0.16,-0.97,0.43
2,2007-04-30,7.153685,13.348787,7.164454,43.265350,9.613921,2.044975,5.077626,4.993695,0.169763,...,12.778380,2.817088,7.561204,4.776457,22.567200,31.585295,3.49,-2.16,-1.45,0.44
3,2007-05-31,19.419975,5.181152,2.474475,11.986431,-0.866247,-0.670016,4.664026,29.199315,3.171925,...,-26.859959,-20.763936,-13.074411,-1.125715,13.226797,-0.417537,3.24,0.24,-0.65,0.41


### Hyperparameter Tuning - Ridge Regression

In [10]:
# Define the Fama-French factor columns
ff_3_factors = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RF']

# Get list of stock columns (everything except the FF factors and 'Period')
stock_columns = [col for col in three_ff_cleaned_df.columns if col not in ff_3_factors]

# Start global timer
start_all = time.time()

# ------------------ Initialize storage ------------------
all_ridge_results = []             # List of dicts with best model info per stock
best_ridge_models = {}             # Dict of best Ridge model per stock

# ------------------ Model Selection Loop ------------------
# Loop through each stock
for idx, stock in enumerate(stock_columns):
    print(f"\n🟠 Ridge - Stock {idx + 1}/{len(stock_columns)}: {stock}")
    start_time = time.time()
    # Subset and process
    df_subset = three_ff_cleaned_df[ff_3_factors + [stock]].copy()
    df_subset['Stock_Return'] = df_subset[[stock]]
    df_subset['Excess_Mkt_Return'] = df_subset['Mkt-RF'] - df_subset['RF']
    df_subset = df_subset[['Date', 'Stock_Return', 'Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
    
    # Shift return behind by 1 to align predictors at t with return at t+1
    df_subset['Stock_Return_Shifted'] = df_subset['Stock_Return'].shift(-1)
    # Drop the first row which now has NaN in 'Stock_Return_Shifted'
    df_subset = df_subset.dropna()

    # Standardize the predictors
    scaler = StandardScaler()
    df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RF']] = scaler.fit_transform(df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RF']])

    ridge_results = []
    alphas = [0.01, 0.1, 1.0, 10.0, 100.0]

    # loop each hyperparameter for each stock
    for alpha in alphas:

        model = Ridge(alpha=alpha, random_state = random_seed)

        val_rmse_list = []

        for i in range(train_size, train_size + val_size - 3):
            train_end_month = month_periods[i - 1]
            val_start_month = month_periods[i]
            val_end_month = month_periods[i + 2]
            
#             print("train_end_month")
#             print(train_end_month)
#             print("val_start_month")
#             print(val_start_month)
#             print("val_end_month")
#             print(val_end_month)

            train_mask = df_subset['Date'].dt.to_period('M') <= train_end_month
            val_mask = (df_subset['Date'].dt.to_period('M') >= val_start_month) & \
                       (df_subset['Date'].dt.to_period('M') <= val_end_month)

            X_train = df_subset.loc[train_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
            y_train = df_subset.loc[train_mask, 'Stock_Return_Shifted']
            X_val = df_subset.loc[val_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
            y_val = df_subset.loc[val_mask, 'Stock_Return_Shifted']

            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)

            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            val_rmse_list.append(rmse)

        avg_val_rmse = np.mean(val_rmse_list)

        ridge_results.append({
            'Stock': stock,
            'Alpha': alpha,
            'Avg_Validation_RMSE': avg_val_rmse,
            'Model': model
        })

    # Sort and pick best model for this stock
    ridge_results = sorted(ridge_results, key=lambda x: x['Avg_Validation_RMSE'])
    best_result = ridge_results[0] #lowest RMSE

    # Store best model and result
    all_ridge_results.append(best_result)
    best_ridge_models[stock] = best_result['Model']
    
    elapsed = time.time() - start_time
    print(f"✅ Finished stock {stock} | Time: {elapsed:.2f}s")

# End global timer
end_all = time.time()
print(f"\n🟩 Finished all stocks | Total Time: {end_all - start_all:.2f} seconds")


🟠 Ridge - Stock 1/975: AAPL.O
✅ Finished stock AAPL.O | Time: 0.21s

🟠 Ridge - Stock 2/975: NVDA.O
✅ Finished stock NVDA.O | Time: 0.23s

🟠 Ridge - Stock 3/975: MSFT.O
✅ Finished stock MSFT.O | Time: 0.23s

🟠 Ridge - Stock 4/975: AMZN.O
✅ Finished stock AMZN.O | Time: 0.27s

🟠 Ridge - Stock 5/975: LLY
✅ Finished stock LLY | Time: 0.23s

🟠 Ridge - Stock 6/975: WMT
✅ Finished stock WMT | Time: 0.23s

🟠 Ridge - Stock 7/975: XOM
✅ Finished stock XOM | Time: 0.22s

🟠 Ridge - Stock 8/975: MA
✅ Finished stock MA | Time: 0.24s

🟠 Ridge - Stock 9/975: UNH
✅ Finished stock UNH | Time: 0.23s

🟠 Ridge - Stock 10/975: ORCL.K
✅ Finished stock ORCL.K | Time: 0.22s

🟠 Ridge - Stock 11/975: COST.O
✅ Finished stock COST.O | Time: 0.23s

🟠 Ridge - Stock 12/975: NFLX.O
✅ Finished stock NFLX.O | Time: 0.26s

🟠 Ridge - Stock 13/975: HD
✅ Finished stock HD | Time: 0.21s

🟠 Ridge - Stock 14/975: CVX
✅ Finished stock CVX | Time: 0.23s

🟠 Ridge - Stock 15/975: CRM
✅ Finished stock CRM | Time: 0.25s

🟠 Ridge - 

✅ Finished stock EQT | Time: 0.29s

🟠 Ridge - Stock 124/975: ROK
✅ Finished stock ROK | Time: 0.29s

🟠 Ridge - Stock 125/975: MPWR.O
✅ Finished stock MPWR.O | Time: 0.30s

🟠 Ridge - Stock 126/975: MCHP.O
✅ Finished stock MCHP.O | Time: 0.28s

🟠 Ridge - Stock 127/975: ANSS.O
✅ Finished stock ANSS.O | Time: 0.28s

🟠 Ridge - Stock 128/975: DTE
✅ Finished stock DTE | Time: 0.29s

🟠 Ridge - Stock 129/975: DXCM.O
✅ Finished stock DXCM.O | Time: 0.32s

🟠 Ridge - Stock 130/975: IP
✅ Finished stock IP | Time: 0.28s

🟠 Ridge - Stock 131/975: AEE
✅ Finished stock AEE | Time: 0.30s

🟠 Ridge - Stock 132/975: PPG
✅ Finished stock PPG | Time: 0.28s

🟠 Ridge - Stock 133/975: PPL
✅ Finished stock PPL | Time: 0.30s

🟠 Ridge - Stock 134/975: MTD
✅ Finished stock MTD | Time: 0.29s

🟠 Ridge - Stock 135/975: WBD.O
✅ Finished stock WBD.O | Time: 0.35s

🟠 Ridge - Stock 136/975: TYL
✅ Finished stock TYL | Time: 0.28s

🟠 Ridge - Stock 137/975: EL
✅ Finished stock EL | Time: 0.29s

🟠 Ridge - Stock 138/975: ATO
✅

✅ Finished stock X | Time: 0.26s

🟠 Ridge - Stock 244/975: HSIC.O
✅ Finished stock HSIC.O | Time: 0.27s

🟠 Ridge - Stock 245/975: LNW.O
✅ Finished stock LNW.O | Time: 0.27s

🟠 Ridge - Stock 246/975: CHE
✅ Finished stock CHE | Time: 0.29s

🟠 Ridge - Stock 247/975: CRL
✅ Finished stock CRL | Time: 0.27s

🟠 Ridge - Stock 248/975: DRS.O
✅ Finished stock DRS.O | Time: 0.33s

🟠 Ridge - Stock 249/975: RGEN.O
✅ Finished stock RGEN.O | Time: 0.28s

🟠 Ridge - Stock 250/975: LSCC.O
✅ Finished stock LSCC.O | Time: 0.42s

🟠 Ridge - Stock 251/975: EXAS.O
✅ Finished stock EXAS.O | Time: 0.44s

🟠 Ridge - Stock 252/975: HAS.O
✅ Finished stock HAS.O | Time: 0.36s

🟠 Ridge - Stock 253/975: PARA.O
✅ Finished stock PARA.O | Time: 0.54s

🟠 Ridge - Stock 254/975: DCI
✅ Finished stock DCI | Time: 0.56s

🟠 Ridge - Stock 255/975: CHDN.O
✅ Finished stock CHDN.O | Time: 0.41s

🟠 Ridge - Stock 256/975: MKTX.O
✅ Finished stock MKTX.O | Time: 0.47s

🟠 Ridge - Stock 257/975: NYT
✅ Finished stock NYT | Time: 0.51s

🟠 

✅ Finished stock ANF | Time: 0.41s

🟠 Ridge - Stock 363/975: JWN
✅ Finished stock JWN | Time: 0.39s

🟠 Ridge - Stock 364/975: ITGR.K
✅ Finished stock ITGR.K | Time: 0.52s

🟠 Ridge - Stock 365/975: SPR
✅ Finished stock SPR | Time: 0.50s

🟠 Ridge - Stock 366/975: DORM.O
✅ Finished stock DORM.O | Time: 0.55s

🟠 Ridge - Stock 367/975: ALE
✅ Finished stock ALE | Time: 0.56s

🟠 Ridge - Stock 368/975: MMS
✅ Finished stock MMS | Time: 0.60s

🟠 Ridge - Stock 369/975: FIZZ.O
✅ Finished stock FIZZ.O | Time: 0.61s

🟠 Ridge - Stock 370/975: TDS
✅ Finished stock TDS | Time: 0.60s

🟠 Ridge - Stock 371/975: CBZ
✅ Finished stock CBZ | Time: 0.56s

🟠 Ridge - Stock 372/975: BCO
✅ Finished stock BCO | Time: 0.72s

🟠 Ridge - Stock 373/975: AVNT.K
✅ Finished stock AVNT.K | Time: 0.48s

🟠 Ridge - Stock 374/975: AVAV.O
✅ Finished stock AVAV.O | Time: 0.43s

🟠 Ridge - Stock 375/975: IESC.O
✅ Finished stock IESC.O | Time: 0.43s

🟠 Ridge - Stock 376/975: M
✅ Finished stock M | Time: 0.43s

🟠 Ridge - Stock 377/97

✅ Finished stock STGW.O | Time: 0.53s

🟠 Ridge - Stock 481/975: WKC
✅ Finished stock WKC | Time: 0.49s

🟠 Ridge - Stock 482/975: MNKD.O
✅ Finished stock MNKD.O | Time: 0.39s

🟠 Ridge - Stock 483/975: MCRI.O
✅ Finished stock MCRI.O | Time: 0.42s

🟠 Ridge - Stock 484/975: APLD.O
✅ Finished stock APLD.O | Time: 0.39s

🟠 Ridge - Stock 485/975: SRCE.O
✅ Finished stock SRCE.O | Time: 0.42s

🟠 Ridge - Stock 486/975: OMCL.O
✅ Finished stock OMCL.O | Time: 0.49s

🟠 Ridge - Stock 487/975: INOD.O
✅ Finished stock INOD.O | Time: 0.36s

🟠 Ridge - Stock 488/975: FL
✅ Finished stock FL | Time: 0.40s

🟠 Ridge - Stock 489/975: NTCT.O
✅ Finished stock NTCT.O | Time: 0.43s

🟠 Ridge - Stock 490/975: EPC
✅ Finished stock EPC | Time: 0.35s

🟠 Ridge - Stock 491/975: ATSG.O
✅ Finished stock ATSG.O | Time: 0.35s

🟠 Ridge - Stock 492/975: ADEA.O
✅ Finished stock ADEA.O | Time: 0.37s

🟠 Ridge - Stock 493/975: ROG
✅ Finished stock ROG | Time: 0.38s

🟠 Ridge - Stock 494/975: NHC
✅ Finished stock NHC | Time: 0.37s


✅ Finished stock OPY | Time: 0.34s

🟠 Ridge - Stock 598/975: REPX.K
✅ Finished stock REPX.K | Time: 0.35s

🟠 Ridge - Stock 599/975: CVLG.K
✅ Finished stock CVLG.K | Time: 0.34s

🟠 Ridge - Stock 600/975: OSPN.O
✅ Finished stock OSPN.O | Time: 0.34s

🟠 Ridge - Stock 601/975: FWRD.O
✅ Finished stock FWRD.O | Time: 0.33s

🟠 Ridge - Stock 602/975: SCVL.O
✅ Finished stock SCVL.O | Time: 0.32s

🟠 Ridge - Stock 603/975: JACK.O
✅ Finished stock JACK.O | Time: 0.33s

🟠 Ridge - Stock 604/975: SENEA.O
✅ Finished stock SENEA.O | Time: 0.34s

🟠 Ridge - Stock 605/975: WLDN.O
✅ Finished stock WLDN.O | Time: 0.33s

🟠 Ridge - Stock 606/975: CASS.O
✅ Finished stock CASS.O | Time: 0.33s

🟠 Ridge - Stock 607/975: SMP
✅ Finished stock SMP | Time: 0.32s

🟠 Ridge - Stock 608/975: CTLP.O
✅ Finished stock CTLP.O | Time: 0.34s

🟠 Ridge - Stock 609/975: FARO.O
✅ Finished stock FARO.O | Time: 0.32s

🟠 Ridge - Stock 610/975: EBF
✅ Finished stock EBF | Time: 0.32s

🟠 Ridge - Stock 611/975: USNA.K
✅ Finished stock US

✅ Finished stock RGCO.O | Time: 0.46s

🟠 Ridge - Stock 715/975: TSSI.O
✅ Finished stock TSSI.O | Time: 0.41s

🟠 Ridge - Stock 716/975: LFVN.O
✅ Finished stock LFVN.O | Time: 0.42s

🟠 Ridge - Stock 717/975: ESCA.O
✅ Finished stock ESCA.O | Time: 0.54s

🟠 Ridge - Stock 718/975: SGMO.O
✅ Finished stock SGMO.O | Time: 0.51s

🟠 Ridge - Stock 719/975: MPAA.O
✅ Finished stock MPAA.O | Time: 0.45s

🟠 Ridge - Stock 720/975: DENN.O
✅ Finished stock DENN.O | Time: 0.46s

🟠 Ridge - Stock 721/975: FXNC.O
✅ Finished stock FXNC.O | Time: 0.41s

🟠 Ridge - Stock 722/975: SWKH.O
✅ Finished stock SWKH.O | Time: 0.34s

🟠 Ridge - Stock 723/975: UTMD.O
✅ Finished stock UTMD.O | Time: 0.38s

🟠 Ridge - Stock 724/975: LAKE.O
✅ Finished stock LAKE.O | Time: 0.42s

🟠 Ridge - Stock 725/975: GENC.K
✅ Finished stock GENC.K | Time: 0.37s

🟠 Ridge - Stock 726/975: LTBR.O
✅ Finished stock LTBR.O | Time: 0.37s

🟠 Ridge - Stock 727/975: HQI.O
✅ Finished stock HQI.O | Time: 0.38s

🟠 Ridge - Stock 728/975: STRT.O
✅ Finish

✅ Finished stock FKWL.O | Time: 0.36s

🟠 Ridge - Stock 830/975: DOMH.O
✅ Finished stock DOMH.O | Time: 0.35s

🟠 Ridge - Stock 831/975: DLHC.O
✅ Finished stock DLHC.O | Time: 0.37s

🟠 Ridge - Stock 832/975: PED
✅ Finished stock PED | Time: 0.39s

🟠 Ridge - Stock 833/975: ALTS.O
✅ Finished stock ALTS.O | Time: 0.40s

🟠 Ridge - Stock 834/975: CULP.K
✅ Finished stock CULP.K | Time: 0.42s

🟠 Ridge - Stock 835/975: SUP
✅ Finished stock SUP | Time: 0.42s

🟠 Ridge - Stock 836/975: CATO.K
✅ Finished stock CATO.K | Time: 0.40s

🟠 Ridge - Stock 837/975: CYCC.O
✅ Finished stock CYCC.O | Time: 0.36s

🟠 Ridge - Stock 838/975: ICAD.O
✅ Finished stock ICAD.O | Time: 0.40s

🟠 Ridge - Stock 839/975: WFCF.O
✅ Finished stock WFCF.O | Time: 0.36s

🟠 Ridge - Stock 840/975: LINK.O
✅ Finished stock LINK.O | Time: 0.37s

🟠 Ridge - Stock 841/975: CTSO.O
✅ Finished stock CTSO.O | Time: 0.55s

🟠 Ridge - Stock 842/975: CNVS.O
✅ Finished stock CNVS.O | Time: 0.67s

🟠 Ridge - Stock 843/975: CNTY.O
✅ Finished stock C

✅ Finished stock ARTW.O | Time: 0.34s

🟠 Ridge - Stock 946/975: AIMD.O
✅ Finished stock AIMD.O | Time: 0.33s

🟠 Ridge - Stock 947/975: AIM
✅ Finished stock AIM | Time: 0.34s

🟠 Ridge - Stock 948/975: STRR.O
✅ Finished stock STRR.O | Time: 0.34s

🟠 Ridge - Stock 949/975: SSY
✅ Finished stock SSY | Time: 0.33s

🟠 Ridge - Stock 950/975: GTBP.O
✅ Finished stock GTBP.O | Time: 0.34s

🟠 Ridge - Stock 951/975: SGMA.O
✅ Finished stock SGMA.O | Time: 0.34s

🟠 Ridge - Stock 952/975: OGEN.K
✅ Finished stock OGEN.K | Time: 0.33s

🟠 Ridge - Stock 953/975: RIME.O
✅ Finished stock RIME.O | Time: 0.34s

🟠 Ridge - Stock 954/975: SNGX.O
✅ Finished stock SNGX.O | Time: 0.33s

🟠 Ridge - Stock 955/975: SNOA.O
✅ Finished stock SNOA.O | Time: 0.33s

🟠 Ridge - Stock 956/975: AEMD.O
✅ Finished stock AEMD.O | Time: 0.33s

🟠 Ridge - Stock 957/975: GBR
✅ Finished stock GBR | Time: 0.33s

🟠 Ridge - Stock 958/975: KNW
✅ Finished stock KNW | Time: 0.32s

🟠 Ridge - Stock 959/975: BKYI.O
✅ Finished stock BKYI.O | Time

### Hyperparameter Tuning - Lasso Regression

In [11]:
# Define the Fama-French factor columns
ff_3_factors = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RF']

# Get list of stock columns (everything except the FF factors and 'Period')
stock_columns = [col for col in three_ff_cleaned_df.columns if col not in ff_3_factors]

# Start global timer
start_all = time.time()
# ------------------ Lasso Regression ------------------

all_lasso_results = []
best_lasso_models = {}

for idx, stock in enumerate(stock_columns):
    print(f"\n🔴 Lasso - Stock {idx + 1}/{len(stock_columns)}: {stock}")
    start_time = time.time()

    # Subset and process
    df_subset = three_ff_cleaned_df[ff_3_factors + [stock]].copy()
    df_subset['Stock_Return'] = df_subset[[stock]]
    df_subset['Excess_Mkt_Return'] = df_subset['Mkt-RF'] - df_subset['RF']
    df_subset = df_subset[['Date', 'Stock_Return', 'Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
    
    # Shift return behind by 1 to align predictors at t with return at t+1
    df_subset['Stock_Return_Shifted'] = df_subset['Stock_Return'].shift(-1)

    # Drop the first row which now has NaN in 'Stock_Return_Shifted'
    df_subset = df_subset.dropna()

    # Standardize the predictors
    scaler = StandardScaler()
    df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RF']] = scaler.fit_transform(df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RF']])

    lasso_results = []
    for alpha in alphas:
        model = Lasso(alpha=alpha, random_state=random_seed, max_iter=10000)
        val_rmse_list = []

        for i in range(train_size, train_size + val_size - 3):
            train_end = month_periods[i - 1]
            val_start = month_periods[i]
            val_end = month_periods[i + 2]

            train_mask = df_subset['Date'].dt.to_period('M') <= train_end
            val_mask = (df_subset['Date'].dt.to_period('M') >= val_start) & (df_subset['Date'].dt.to_period('M') <= val_end)

            X_train = df_subset.loc[train_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
            y_train = df_subset.loc[train_mask, 'Stock_Return_Shifted']
            X_val = df_subset.loc[val_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
            y_val = df_subset.loc[val_mask, 'Stock_Return_Shifted']

            model.fit(X_train, y_train)
            rmse = np.sqrt(mean_squared_error(y_val, model.predict(X_val)))
            val_rmse_list.append(rmse)

        lasso_results.append({
            'Stock': stock,
            'Alpha': alpha,
            'Avg_Validation_RMSE': np.mean(val_rmse_list),
            'Model': model
        })

    lasso_results.sort(key=lambda x: x['Avg_Validation_RMSE'])
    best_lasso_models[stock] = lasso_results[0]['Model']
    all_lasso_results.append(lasso_results[0])

    print(f"✅ Lasso done for {stock} | Best alpha: {lasso_results[0]['Alpha']}")
    
# End global timer
end_all = time.time()
print(f"\n🟩 Finished all stocks | Total Time: {end_all - start_all:.2f} seconds")


🔴 Lasso - Stock 1/975: AAPL.O
✅ Lasso done for AAPL.O | Best alpha: 10.0

🔴 Lasso - Stock 2/975: NVDA.O
✅ Lasso done for NVDA.O | Best alpha: 10.0

🔴 Lasso - Stock 3/975: MSFT.O
✅ Lasso done for MSFT.O | Best alpha: 1.0

🔴 Lasso - Stock 4/975: AMZN.O
✅ Lasso done for AMZN.O | Best alpha: 0.01

🔴 Lasso - Stock 5/975: LLY
✅ Lasso done for LLY | Best alpha: 1.0

🔴 Lasso - Stock 6/975: WMT
✅ Lasso done for WMT | Best alpha: 0.01

🔴 Lasso - Stock 7/975: XOM
✅ Lasso done for XOM | Best alpha: 10.0

🔴 Lasso - Stock 8/975: MA
✅ Lasso done for MA | Best alpha: 10.0

🔴 Lasso - Stock 9/975: UNH
✅ Lasso done for UNH | Best alpha: 1.0

🔴 Lasso - Stock 10/975: ORCL.K
✅ Lasso done for ORCL.K | Best alpha: 0.01

🔴 Lasso - Stock 11/975: COST.O
✅ Lasso done for COST.O | Best alpha: 1.0

🔴 Lasso - Stock 12/975: NFLX.O
✅ Lasso done for NFLX.O | Best alpha: 0.1

🔴 Lasso - Stock 13/975: HD
✅ Lasso done for HD | Best alpha: 1.0

🔴 Lasso - Stock 14/975: CVX
✅ Lasso done for CVX | Best alpha: 10.0

🔴 Lasso - 

✅ Lasso done for LVS | Best alpha: 10.0

🔴 Lasso - Stock 117/975: CAH
✅ Lasso done for CAH | Best alpha: 1.0

🔴 Lasso - Stock 118/975: WAB
✅ Lasso done for WAB | Best alpha: 1.0

🔴 Lasso - Stock 119/975: HUM
✅ Lasso done for HUM | Best alpha: 1.0

🔴 Lasso - Stock 120/975: NUE
✅ Lasso done for NUE | Best alpha: 10.0

🔴 Lasso - Stock 121/975: EBAY.O
✅ Lasso done for EBAY.O | Best alpha: 10.0

🔴 Lasso - Stock 122/975: TPL
✅ Lasso done for TPL | Best alpha: 0.01

🔴 Lasso - Stock 123/975: EQT
✅ Lasso done for EQT | Best alpha: 1.0

🔴 Lasso - Stock 124/975: ROK
✅ Lasso done for ROK | Best alpha: 1.0

🔴 Lasso - Stock 125/975: MPWR.O
✅ Lasso done for MPWR.O | Best alpha: 1.0

🔴 Lasso - Stock 126/975: MCHP.O
✅ Lasso done for MCHP.O | Best alpha: 10.0

🔴 Lasso - Stock 127/975: ANSS.O
✅ Lasso done for ANSS.O | Best alpha: 1.0

🔴 Lasso - Stock 128/975: DTE
✅ Lasso done for DTE | Best alpha: 0.01

🔴 Lasso - Stock 129/975: DXCM.O
✅ Lasso done for DXCM.O | Best alpha: 10.0

🔴 Lasso - Stock 130/975: I

✅ Lasso done for DOX.O | Best alpha: 1.0

🔴 Lasso - Stock 230/975: IPG
✅ Lasso done for IPG | Best alpha: 10.0

🔴 Lasso - Stock 231/975: TECH.O
✅ Lasso done for TECH.O | Best alpha: 1.0

🔴 Lasso - Stock 232/975: ATR
✅ Lasso done for ATR | Best alpha: 1.0

🔴 Lasso - Stock 233/975: WBA.O
✅ Lasso done for WBA.O | Best alpha: 0.01

🔴 Lasso - Stock 234/975: MTZ
✅ Lasso done for MTZ | Best alpha: 1.0

🔴 Lasso - Stock 235/975: SRPT.O
✅ Lasso done for SRPT.O | Best alpha: 0.1

🔴 Lasso - Stock 236/975: SAIA.O
✅ Lasso done for SAIA.O | Best alpha: 10.0

🔴 Lasso - Stock 237/975: SEIC.O
✅ Lasso done for SEIC.O | Best alpha: 10.0

🔴 Lasso - Stock 238/975: RRC
✅ Lasso done for RRC | Best alpha: 1.0

🔴 Lasso - Stock 239/975: WYNN.O
✅ Lasso done for WYNN.O | Best alpha: 10.0

🔴 Lasso - Stock 240/975: CRS
✅ Lasso done for CRS | Best alpha: 1.0

🔴 Lasso - Stock 241/975: CIEN.K
✅ Lasso done for CIEN.K | Best alpha: 1.0

🔴 Lasso - Stock 242/975: ALB
✅ Lasso done for ALB | Best alpha: 0.01

🔴 Lasso - Stock

✅ Lasso done for DY | Best alpha: 10.0

🔴 Lasso - Stock 342/975: URBN.O
✅ Lasso done for URBN.O | Best alpha: 10.0

🔴 Lasso - Stock 343/975: PIPR.K
✅ Lasso done for PIPR.K | Best alpha: 10.0

🔴 Lasso - Stock 344/975: MOD
✅ Lasso done for MOD | Best alpha: 10.0

🔴 Lasso - Stock 345/975: EEFT.O
✅ Lasso done for EEFT.O | Best alpha: 1.0

🔴 Lasso - Stock 346/975: BKH
✅ Lasso done for BKH | Best alpha: 10.0

🔴 Lasso - Stock 347/975: KMPR.K
✅ Lasso done for KMPR.K | Best alpha: 0.01

🔴 Lasso - Stock 348/975: KBH
✅ Lasso done for KBH | Best alpha: 10.0

🔴 Lasso - Stock 349/975: MSM
✅ Lasso done for MSM | Best alpha: 1.0

🔴 Lasso - Stock 350/975: THO
✅ Lasso done for THO | Best alpha: 1.0

🔴 Lasso - Stock 351/975: EXPO.O
✅ Lasso done for EXPO.O | Best alpha: 1.0

🔴 Lasso - Stock 352/975: NNI
✅ Lasso done for NNI | Best alpha: 10.0

🔴 Lasso - Stock 353/975: ESE
✅ Lasso done for ESE | Best alpha: 1.0

🔴 Lasso - Stock 354/975: PBH
✅ Lasso done for PBH | Best alpha: 1.0

🔴 Lasso - Stock 355/975: A

✅ Lasso done for LGFa | Best alpha: 10.0

🔴 Lasso - Stock 453/975: LGND.O
✅ Lasso done for LGND.O | Best alpha: 10.0

🔴 Lasso - Stock 454/975: HNI
✅ Lasso done for HNI | Best alpha: 10.0

🔴 Lasso - Stock 455/975: JBLU.O
✅ Lasso done for JBLU.O | Best alpha: 10.0

🔴 Lasso - Stock 456/975: NEOG.O
✅ Lasso done for NEOG.O | Best alpha: 1.0

🔴 Lasso - Stock 457/975: HE
✅ Lasso done for HE | Best alpha: 10.0

🔴 Lasso - Stock 458/975: EXTR.O
✅ Lasso done for EXTR.O | Best alpha: 10.0

🔴 Lasso - Stock 459/975: ACLS.O
✅ Lasso done for ACLS.O | Best alpha: 10.0

🔴 Lasso - Stock 460/975: WERN.O
✅ Lasso done for WERN.O | Best alpha: 1.0

🔴 Lasso - Stock 461/975: LMAT.O
✅ Lasso done for LMAT.O | Best alpha: 1.0

🔴 Lasso - Stock 462/975: CENX.O
✅ Lasso done for CENX.O | Best alpha: 10.0

🔴 Lasso - Stock 463/975: BKE
✅ Lasso done for BKE | Best alpha: 1.0

🔴 Lasso - Stock 464/975: CNMD.K
✅ Lasso done for CNMD.K | Best alpha: 1.0

🔴 Lasso - Stock 465/975: OI
✅ Lasso done for OI | Best alpha: 10.0

🔴 L

✅ Lasso done for STAA.O | Best alpha: 0.1

🔴 Lasso - Stock 563/975: ASTE.O
✅ Lasso done for ASTE.O | Best alpha: 10.0

🔴 Lasso - Stock 564/975: CECO.O
✅ Lasso done for CECO.O | Best alpha: 10.0

🔴 Lasso - Stock 565/975: ECPG.O
✅ Lasso done for ECPG.O | Best alpha: 10.0

🔴 Lasso - Stock 566/975: PRA
✅ Lasso done for PRA | Best alpha: 1.0

🔴 Lasso - Stock 567/975: AVXL.O
✅ Lasso done for AVXL.O | Best alpha: 0.1

🔴 Lasso - Stock 568/975: COHU.O
✅ Lasso done for COHU.O | Best alpha: 1.0

🔴 Lasso - Stock 569/975: AMSC.O
✅ Lasso done for AMSC.O | Best alpha: 10.0

🔴 Lasso - Stock 570/975: HCKT.O
✅ Lasso done for HCKT.O | Best alpha: 10.0

🔴 Lasso - Stock 571/975: AIOT.O
✅ Lasso done for AIOT.O | Best alpha: 1.0

🔴 Lasso - Stock 572/975: PRAA.O
✅ Lasso done for PRAA.O | Best alpha: 0.01

🔴 Lasso - Stock 573/975: AXGN.O
✅ Lasso done for AXGN.O | Best alpha: 1.0

🔴 Lasso - Stock 574/975: OSBC.O
✅ Lasso done for OSBC.O | Best alpha: 10.0

🔴 Lasso - Stock 575/975: UFCS.O
✅ Lasso done for UFCS.O 

✅ Lasso done for NVEC.O | Best alpha: 10.0

🔴 Lasso - Stock 673/975: EBS
✅ Lasso done for EBS | Best alpha: 0.01

🔴 Lasso - Stock 674/975: OIS
✅ Lasso done for OIS | Best alpha: 10.0

🔴 Lasso - Stock 675/975: RLGT.K
✅ Lasso done for RLGT.K | Best alpha: 10.0

🔴 Lasso - Stock 676/975: BYON.K
✅ Lasso done for BYON.K | Best alpha: 10.0

🔴 Lasso - Stock 677/975: MVIS.O
✅ Lasso done for MVIS.O | Best alpha: 10.0

🔴 Lasso - Stock 678/975: UIS
✅ Lasso done for UIS | Best alpha: 10.0

🔴 Lasso - Stock 679/975: CDZI.O
✅ Lasso done for CDZI.O | Best alpha: 10.0

🔴 Lasso - Stock 680/975: VNDA.O
✅ Lasso done for VNDA.O | Best alpha: 10.0

🔴 Lasso - Stock 681/975: NGS
✅ Lasso done for NGS | Best alpha: 0.1

🔴 Lasso - Stock 682/975: EGHT.O
✅ Lasso done for EGHT.O | Best alpha: 10.0

🔴 Lasso - Stock 683/975: PAMT.O
✅ Lasso done for PAMT.O | Best alpha: 0.1

🔴 Lasso - Stock 684/975: FNLC.O
✅ Lasso done for FNLC.O | Best alpha: 0.01

🔴 Lasso - Stock 685/975: AEHR.O
✅ Lasso done for AEHR.O | Best alpha: 

✅ Lasso done for GNSS.O | Best alpha: 10.0

🔴 Lasso - Stock 782/975: GALT.O
✅ Lasso done for GALT.O | Best alpha: 10.0

🔴 Lasso - Stock 783/975: GAIA.O
✅ Lasso done for GAIA.O | Best alpha: 10.0

🔴 Lasso - Stock 784/975: KVHI.O
✅ Lasso done for KVHI.O | Best alpha: 10.0

🔴 Lasso - Stock 785/975: HURC.O
✅ Lasso done for HURC.O | Best alpha: 10.0

🔴 Lasso - Stock 786/975: NOTV.O
✅ Lasso done for NOTV.O | Best alpha: 10.0

🔴 Lasso - Stock 787/975: DXLG.O
✅ Lasso done for DXLG.O | Best alpha: 10.0

🔴 Lasso - Stock 788/975: TAYD.O
✅ Lasso done for TAYD.O | Best alpha: 10.0

🔴 Lasso - Stock 789/975: PCG_pa
✅ Lasso done for PCG_pa | Best alpha: 1.0

🔴 Lasso - Stock 790/975: GEOS.O
✅ Lasso done for GEOS.O | Best alpha: 0.01

🔴 Lasso - Stock 791/975: SLNG.O
✅ Lasso done for SLNG.O | Best alpha: 10.0

🔴 Lasso - Stock 792/975: ORMP.O
✅ Lasso done for ORMP.O | Best alpha: 10.0

🔴 Lasso - Stock 793/975: INTT.K
✅ Lasso done for INTT.K | Best alpha: 10.0

🔴 Lasso - Stock 794/975: ACHV.O
✅ Lasso done 

✅ Lasso done for INTG.O | Best alpha: 0.01

🔴 Lasso - Stock 890/975: DRRX.O
✅ Lasso done for DRRX.O | Best alpha: 10.0

🔴 Lasso - Stock 891/975: IGC
✅ Lasso done for IGC | Best alpha: 10.0

🔴 Lasso - Stock 892/975: TRT
✅ Lasso done for TRT | Best alpha: 10.0

🔴 Lasso - Stock 893/975: ENZ
✅ Lasso done for ENZ | Best alpha: 0.01

🔴 Lasso - Stock 894/975: GTIM.O
✅ Lasso done for GTIM.O | Best alpha: 10.0

🔴 Lasso - Stock 895/975: TENX.O
✅ Lasso done for TENX.O | Best alpha: 10.0

🔴 Lasso - Stock 896/975: CKX
✅ Lasso done for CKX | Best alpha: 0.1

🔴 Lasso - Stock 897/975: CRIS.O
✅ Lasso done for CRIS.O | Best alpha: 10.0

🔴 Lasso - Stock 898/975: CVM
✅ Lasso done for CVM | Best alpha: 10.0

🔴 Lasso - Stock 899/975: CYTH.O
✅ Lasso done for CYTH.O | Best alpha: 10.0

🔴 Lasso - Stock 900/975: DAIO.O
✅ Lasso done for DAIO.O | Best alpha: 10.0

🔴 Lasso - Stock 901/975: JOB
✅ Lasso done for JOB | Best alpha: 10.0

🔴 Lasso - Stock 902/975: RVP
✅ Lasso done for RVP | Best alpha: 0.01

🔴 Lasso - S

### Hyperparameter Tuning - ElasticNet

In [12]:
# Define the Fama-French factor columns
ff_3_factors = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RF']

# Get list of stock columns (everything except the FF factors and 'Period')
stock_columns = [col for col in three_ff_cleaned_df.columns if col not in ff_3_factors]

# Start global timer
start_all = time.time()

# ------------------ ElasticNet Regression ------------------

all_elastic_results = []
best_elastic_models = {}

for idx, stock in enumerate(stock_columns):
    print(f"\n🟠 ElasticNet - Stock {idx + 1}/{len(stock_columns)}: {stock}")
    start_time = time.time()

    # Subset and process
    df_subset = three_ff_cleaned_df[ff_3_factors + [stock]].copy()
    df_subset['Stock_Return'] = df_subset[[stock]]
    df_subset['Excess_Mkt_Return'] = df_subset['Mkt-RF'] - df_subset['RF']
    df_subset = df_subset[['Date', 'Stock_Return', 'Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
    
    # Shift return behind by 1 to align predictors at t with return at t+1
    df_subset['Stock_Return_Shifted'] = df_subset['Stock_Return'].shift(-1)

    # Drop the first row which now has NaN in 'Stock_Return_Shifted'
    df_subset = df_subset.dropna()

    # Standardize the predictors
    scaler = StandardScaler()
    df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RF']] = scaler.fit_transform(df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RF']])

    elastic_results = []
    for alpha in alphas:
        for l1_ratio in l1_ratios:
            model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=random_seed, max_iter=10000)
            val_rmse_list = []

            for i in range(train_size, train_size + val_size - 3):
                train_end = month_periods[i - 1]
                val_start = month_periods[i]
                val_end = month_periods[i + 2]

                train_mask = df_subset['Date'].dt.to_period('M') <= train_end
                val_mask = (df_subset['Date'].dt.to_period('M') >= val_start) & (df_subset['Date'].dt.to_period('M') <= val_end)

                X_train = df_subset.loc[train_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
                y_train = df_subset.loc[train_mask, 'Stock_Return_Shifted']
                X_val = df_subset.loc[val_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
                y_val = df_subset.loc[val_mask, 'Stock_Return_Shifted']

                model.fit(X_train, y_train)
                rmse = np.sqrt(mean_squared_error(y_val, model.predict(X_val)))
                val_rmse_list.append(rmse)

            elastic_results.append({
                'Stock': stock,
                'Alpha': alpha,
                'L1_Ratio': l1_ratio,
                'Avg_Validation_RMSE': np.mean(val_rmse_list),
                'Model': model
            })

    elastic_results.sort(key=lambda x: x['Avg_Validation_RMSE'])
    best_elastic_models[stock] = elastic_results[0]['Model']
    all_elastic_results.append(elastic_results[0])
    print(f"✅ ElasticNet done for {stock} | Best α: {elastic_results[0]['Alpha']} | L1: {elastic_results[0]['L1_Ratio']}")
    
# End global timer
end_all = time.time()
print(f"\n🟩 Finished all stocks | Total Time: {end_all - start_all:.2f} seconds")


🟠 ElasticNet - Stock 1/975: AAPL.O
✅ ElasticNet done for AAPL.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 2/975: NVDA.O
✅ ElasticNet done for NVDA.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 3/975: MSFT.O
✅ ElasticNet done for MSFT.O | Best α: 1.0 | L1: 0.9

🟠 ElasticNet - Stock 4/975: AMZN.O
✅ ElasticNet done for AMZN.O | Best α: 0.1 | L1: 0.1

🟠 ElasticNet - Stock 5/975: LLY
✅ ElasticNet done for LLY | Best α: 1.0 | L1: 0.9

🟠 ElasticNet - Stock 6/975: WMT
✅ ElasticNet done for WMT | Best α: 0.01 | L1: 0.9

🟠 ElasticNet - Stock 7/975: XOM
✅ ElasticNet done for XOM | Best α: 1.0 | L1: 0.1

🟠 ElasticNet - Stock 8/975: MA
✅ ElasticNet done for MA | Best α: 1.0 | L1: 0.5

🟠 ElasticNet - Stock 9/975: UNH
✅ ElasticNet done for UNH | Best α: 1.0 | L1: 0.1

🟠 ElasticNet - Stock 10/975: ORCL.K
✅ ElasticNet done for ORCL.K | Best α: 0.1 | L1: 0.1

🟠 ElasticNet - Stock 11/975: COST.O
✅ ElasticNet done for COST.O | Best α: 1.0 | L1: 0.5

🟠 ElasticNet - Stock 12/975: NFLX.O
✅ ElasticNe

✅ ElasticNet done for OXY | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 96/975: BKR.O
✅ ElasticNet done for BKR.O | Best α: 1.0 | L1: 0.5

🟠 ElasticNet - Stock 97/975: ROST.O
✅ ElasticNet done for ROST.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 98/975: EW
✅ ElasticNet done for EW | Best α: 1.0 | L1: 0.9

🟠 ElasticNet - Stock 99/975: VLO
✅ ElasticNet done for VLO | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 100/975: AME
✅ ElasticNet done for AME | Best α: 1.0 | L1: 0.9

🟠 ElasticNet - Stock 101/975: GLW
✅ ElasticNet done for GLW | Best α: 1.0 | L1: 0.1

🟠 ElasticNet - Stock 102/975: DHI
✅ ElasticNet done for DHI | Best α: 10.0 | L1: 0.1

🟠 ElasticNet - Stock 103/975: LHX
✅ ElasticNet done for LHX | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 104/975: CTSH.O
✅ ElasticNet done for CTSH.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 105/975: PWR
✅ ElasticNet done for PWR | Best α: 1.0 | L1: 0.5

🟠 ElasticNet - Stock 106/975: F
✅ ElasticNet done for F | Best α: 0.1 | L1: 0.1

🟠 

✅ ElasticNet done for DLTR.O | Best α: 0.1 | L1: 0.9

🟠 ElasticNet - Stock 189/975: INSM.O
✅ ElasticNet done for INSM.O | Best α: 10.0 | L1: 0.1

🟠 ElasticNet - Stock 190/975: UTHR.O
✅ ElasticNet done for UTHR.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 191/975: HOLX.O
✅ ElasticNet done for HOLX.O | Best α: 1.0 | L1: 0.1

🟠 ElasticNet - Stock 192/975: ILMN.O
✅ ElasticNet done for ILMN.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 193/975: RVTY.K
✅ ElasticNet done for RVTY.K | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 194/975: BMRN.O
✅ ElasticNet done for BMRN.O | Best α: 1.0 | L1: 0.9

🟠 ElasticNet - Stock 195/975: FLEX.O
✅ ElasticNet done for FLEX.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 196/975: INCY.O
✅ ElasticNet done for INCY.O | Best α: 1.0 | L1: 0.9

🟠 ElasticNet - Stock 197/975: JKHY.O
✅ ElasticNet done for JKHY.O | Best α: 0.01 | L1: 0.1

🟠 ElasticNet - Stock 198/975: MORN.O
✅ ElasticNet done for MORN.O | Best α: 1.0 | L1: 0.9

🟠 ElasticNet - Stock 199/975: 

✅ ElasticNet done for QRVO.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 281/975: AAON.O
✅ ElasticNet done for AAON.O | Best α: 1.0 | L1: 0.9

🟠 ElasticNet - Stock 282/975: ALK
✅ ElasticNet done for ALK | Best α: 1.0 | L1: 0.9

🟠 ElasticNet - Stock 283/975: UFPI.O
✅ ElasticNet done for UFPI.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 284/975: FLS
✅ ElasticNet done for FLS | Best α: 1.0 | L1: 0.9

🟠 ElasticNet - Stock 285/975: SNV
✅ ElasticNet done for SNV | Best α: 10.0 | L1: 0.1

🟠 ElasticNet - Stock 286/975: TGTX.O
✅ ElasticNet done for TGTX.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 287/975: TFX
✅ ElasticNet done for TFX | Best α: 1.0 | L1: 0.9

🟠 ElasticNet - Stock 288/975: CELH.O
✅ ElasticNet done for CELH.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 289/975: EAT
✅ ElasticNet done for EAT | Best α: 10.0 | L1: 0.1

🟠 ElasticNet - Stock 290/975: PEGA.O
✅ ElasticNet done for PEGA.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 291/975: OSK
✅ ElasticNet done for OSK

✅ ElasticNet done for BCO | Best α: 1.0 | L1: 0.1

🟠 ElasticNet - Stock 373/975: AVNT.K
✅ ElasticNet done for AVNT.K | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 374/975: AVAV.O
✅ ElasticNet done for AVAV.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 375/975: IESC.O
✅ ElasticNet done for IESC.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 376/975: M
✅ ElasticNet done for M | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 377/975: PRIM.K
✅ ElasticNet done for PRIM.K | Best α: 10.0 | L1: 0.1

🟠 ElasticNet - Stock 378/975: SKYW.O
✅ ElasticNet done for SKYW.O | Best α: 1.0 | L1: 0.1

🟠 ElasticNet - Stock 379/975: ICUI.O
✅ ElasticNet done for ICUI.O | Best α: 0.01 | L1: 0.9

🟠 ElasticNet - Stock 380/975: KFY
✅ ElasticNet done for KFY | Best α: 1.0 | L1: 0.1

🟠 ElasticNet - Stock 381/975: RDNT.O
✅ ElasticNet done for RDNT.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 382/975: NWE.O
✅ ElasticNet done for NWE.O | Best α: 1.0 | L1: 0.9

🟠 ElasticNet - Stock 383/975: SM
✅ ElasticNet done

...

✅ ElasticNet done for KSS | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 555/975: CBRL.O
✅ ElasticNet done for CBRL.O | Best α: 0.01 | L1: 0.1

🟠 ElasticNet - Stock 556/975: DCO
✅ ElasticNet done for DCO | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 557/975: ATRO.O
✅ ElasticNet done for ATRO.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 558/975: HSII.O
✅ ElasticNet done for HSII.O | Best α: 1.0 | L1: 0.1

🟠 ElasticNet - Stock 559/975: GIC
✅ ElasticNet done for GIC | Best α: 10.0 | L1: 0.1

🟠 ElasticNet - Stock 560/975: PDFS.O
✅ ElasticNet done for PDFS.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 561/975: SCSC.O
✅ ElasticNet done for SCSC.O | Best α: 1.0 | L1: 0.9

🟠 ElasticNet - Stock 562/975: STAA.O
✅ ElasticNet done for STAA.O | Best α: 0.1 | L1: 0.9

🟠 ElasticNet - Stock 563/975: ASTE.O
✅ ElasticNet done for ASTE.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 564/975: CECO.O
✅ ElasticNet done for CECO.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 565/975: ECPG.O
✅ Elas

✅ ElasticNet done for SRDX.O | Best α: 1.0 | L1: 0.5

🟠 ElasticNet - Stock 645/975: FC
✅ ElasticNet done for FC | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 646/975: NUS
✅ ElasticNet done for NUS | Best α: 0.01 | L1: 0.9

🟠 ElasticNet - Stock 647/975: GCBC.O
✅ ElasticNet done for GCBC.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 648/975: TBRG.O
✅ ElasticNet done for TBRG.O | Best α: 0.01 | L1: 0.9

🟠 ElasticNet - Stock 649/975: SIGA.O
✅ ElasticNet done for SIGA.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 650/975: ALNT.O
✅ ElasticNet done for ALNT.O | Best α: 1.0 | L1: 0.1

🟠 ElasticNet - Stock 651/975: DIN
✅ ElasticNet done for DIN | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 652/975: NATH.O
✅ ElasticNet done for NATH.O | Best α: 10.0 | L1: 0.1

🟠 ElasticNet - Stock 653/975: VALU.O
✅ ElasticNet done for VALU.O | Best α: 0.1 | L1: 0.1

🟠 ElasticNet - Stock 654/975: ZEUS.O
✅ ElasticNet done for ZEUS.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 655/975: ANGO.O
✅ ElasticNe

✅ ElasticNet done for TBI | Best α: 0.1 | L1: 0.9

🟠 ElasticNet - Stock 735/975: EPM
✅ ElasticNet done for EPM | Best α: 10.0 | L1: 0.1

🟠 ElasticNet - Stock 736/975: VOXX.O
✅ ElasticNet done for VOXX.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 737/975: DBI
✅ ElasticNet done for DBI | Best α: 1.0 | L1: 0.5

🟠 ElasticNet - Stock 738/975: NKTR.O
✅ ElasticNet done for NKTR.O | Best α: 1.0 | L1: 0.1

🟠 ElasticNet - Stock 739/975: PLX
✅ ElasticNet done for PLX | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 740/975: TTEC.O
✅ ElasticNet done for TTEC.O | Best α: 0.01 | L1: 0.9

🟠 ElasticNet - Stock 741/975: PDEX.O
✅ ElasticNet done for PDEX.O | Best α: 10.0 | L1: 0.1

🟠 ElasticNet - Stock 742/975: VIRC.O
✅ ElasticNet done for VIRC.O | Best α: 10.0 | L1: 0.1

🟠 ElasticNet - Stock 743/975: ESOA.O
✅ ElasticNet done for ESOA.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 744/975: CXDO.O
✅ ElasticNet done for CXDO.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 745/975: CSPI.O
✅ ElasticNet

✅ ElasticNet done for AREN.K | Best α: 0.01 | L1: 0.1

🟠 ElasticNet - Stock 825/975: VERU.O
✅ ElasticNet done for VERU.O | Best α: 0.1 | L1: 0.1

🟠 ElasticNet - Stock 826/975: ASYS.O
✅ ElasticNet done for ASYS.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 827/975: ARMP.K
✅ ElasticNet done for ARMP.K | Best α: 1.0 | L1: 0.1

🟠 ElasticNet - Stock 828/975: ASRT.O
✅ ElasticNet done for ASRT.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 829/975: FKWL.O
✅ ElasticNet done for FKWL.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 830/975: DOMH.O
✅ ElasticNet done for DOMH.O | Best α: 1.0 | L1: 0.5

🟠 ElasticNet - Stock 831/975: DLHC.O
✅ ElasticNet done for DLHC.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 832/975: PED
✅ ElasticNet done for PED | Best α: 1.0 | L1: 0.1

🟠 ElasticNet - Stock 833/975: ALTS.O
✅ ElasticNet done for ALTS.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 834/975: CULP.K
✅ ElasticNet done for CULP.K | Best α: 1.0 | L1: 0.5

🟠 ElasticNet - Stock 835/975: SUP
✅ 

✅ ElasticNet done for SIF | Best α: 10.0 | L1: 0.1

🟠 ElasticNet - Stock 915/975: BRN
✅ ElasticNet done for BRN | Best α: 10.0 | L1: 0.1

🟠 ElasticNet - Stock 916/975: NEPH.O
✅ ElasticNet done for NEPH.O | Best α: 10.0 | L1: 0.9

🟠 ElasticNet - Stock 917/975: HUSA.K
✅ ElasticNet done for HUSA.K | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 918/975: CLRO.O
✅ ElasticNet done for CLRO.O | Best α: 10.0 | L1: 0.5

🟠 ElasticNet - Stock 919/975: COHN.K
✅ ElasticNet done for COHN.K | Best α: 1.0 | L1: 0.1

🟠 ElasticNet - Stock 920/975: ELSE.O
✅ ElasticNet done for ELSE.O | Best α: 1.0 | L1: 0.1

🟠 ElasticNet - Stock 921/975: MEIP.O
✅ ElasticNet done for MEIP.O | Best α: 0.01 | L1: 0.1

🟠 ElasticNet - Stock 922/975: TAIT.O
✅ ElasticNet done for TAIT.O | Best α: 1.0 | L1: 0.5

🟠 ElasticNet - Stock 923/975: JCTC.O
✅ ElasticNet done for JCTC.O | Best α: 1.0 | L1: 0.9

🟠 ElasticNet - Stock 924/975: CLRB.O
✅ ElasticNet done for CLRB.O | Best α: 0.01 | L1: 0.9

🟠 ElasticNet - Stock 925/975: GOVX.O
✅

### Hyperparameter Tuning - SVR

In [13]:
# Define the Fama-French factor columns
ff_3_factors = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RF']

# Get list of stock columns (everything except the FF factors and 'Period')
stock_columns = [col for col in three_ff_cleaned_df.columns if col not in ff_3_factors]

# Start global timer
start_all = time.time()

# ------------------ SVR ------------------

all_svr_results = []
best_svr_models = {}

for idx, stock in enumerate(stock_columns):
    print(f"\n🟣 SVR - Stock {idx + 1}/{len(stock_columns)}: {stock}")

    # Subset and process
    df_subset = three_ff_cleaned_df[ff_3_factors + [stock]].copy()
    df_subset['Stock_Return'] = df_subset[[stock]]
    df_subset['Excess_Mkt_Return'] = df_subset['Mkt-RF'] - df_subset['RF']
    df_subset = df_subset[['Date', 'Stock_Return', 'Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
    
    # Shift return behind by 1 to align predictors at t with return at t+1
    df_subset['Stock_Return_Shifted'] = df_subset['Stock_Return'].shift(-1)

    # Drop the first row which now has NaN in 'Stock_Return_Shifted'
    df_subset = df_subset.dropna()

    # Standardize the predictors
    scaler = StandardScaler()
    df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RF']] = scaler.fit_transform(df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RF']])

    svr_results = []

    for C in svr_C_values:
        for epsilon in svr_epsilons:
            model = SVR(C=C, epsilon=epsilon)
            val_rmse_list = []

            for i in range(train_size, train_size + val_size - 3):
                train_end = month_periods[i - 1]
                val_start = month_periods[i]
                val_end = month_periods[i + 2]

                train_mask = df_subset['Date'].dt.to_period('M') <= train_end
                val_mask = (df_subset['Date'].dt.to_period('M') >= val_start) & (df_subset['Date'].dt.to_period('M') <= val_end)

                X_train = df_subset.loc[train_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
                y_train = df_subset.loc[train_mask, 'Stock_Return_Shifted']
                X_val = df_subset.loc[val_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
                y_val = df_subset.loc[val_mask, 'Stock_Return_Shifted']

                model.fit(X_train, y_train)
                rmse = np.sqrt(mean_squared_error(y_val, model.predict(X_val)))
                val_rmse_list.append(rmse)

            svr_results.append({
                'Stock': stock,
                'C': C,
                'Epsilon': epsilon,
                'Avg_Validation_RMSE': np.mean(val_rmse_list),
                'Model': model
            })

    svr_results.sort(key=lambda x: x['Avg_Validation_RMSE'])
    best_svr_models[stock] = svr_results[0]['Model']
    all_svr_results.append(svr_results[0])
    print(f"✅ SVR done for {stock} | Best C: {svr_results[0]['C']} | Epsilon: {svr_results[0]['Epsilon']}")
    
# End global timer
end_all = time.time()
print(f"\n🟩 Finished all stocks | Total Time: {end_all - start_all:.2f} seconds")


🟣 SVR - Stock 1/975: AAPL.O
✅ SVR done for AAPL.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 2/975: NVDA.O
✅ SVR done for NVDA.O | Best C: 1 | Epsilon: 0.01

🟣 SVR - Stock 3/975: MSFT.O
✅ SVR done for MSFT.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 4/975: AMZN.O
✅ SVR done for AMZN.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 5/975: LLY
✅ SVR done for LLY | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 6/975: WMT
✅ SVR done for WMT | Best C: 1 | Epsilon: 0.1

🟣 SVR - Stock 7/975: XOM
✅ SVR done for XOM | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 8/975: MA
✅ SVR done for MA | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 9/975: UNH
✅ SVR done for UNH | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 10/975: ORCL.K
✅ SVR done for ORCL.K | Best C: 1 | Epsilon: 0.1

🟣 SVR - Stock 11/975: COST.O
✅ SVR done for COST.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 12/975: NFLX.O
✅ SVR done for NFLX.O | Best C: 10 | Epsilon: 0.01

🟣 SVR - Stock 13/975: HD
✅ SVR done for HD | Best C: 10 | Epsilon: 0.0

✅ SVR done for F | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 107/975: EA.O
✅ SVR done for EA.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 108/975: TTWO.O
✅ SVR done for TTWO.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 109/975: A
✅ SVR done for A | Best C: 10 | Epsilon: 0.01

🟣 SVR - Stock 110/975: ODFL.O
✅ SVR done for ODFL.O | Best C: 10 | Epsilon: 0.1

🟣 SVR - Stock 111/975: BRO
✅ SVR done for BRO | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 112/975: IDXX.O
✅ SVR done for IDXX.O | Best C: 10 | Epsilon: 0.1

🟣 SVR - Stock 113/975: CSGP.O
✅ SVR done for CSGP.O | Best C: 10 | Epsilon: 0.01

🟣 SVR - Stock 114/975: ALNY.O
✅ SVR done for ALNY.O | Best C: 10 | Epsilon: 0.1

🟣 SVR - Stock 115/975: HEI
✅ SVR done for HEI | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 116/975: LVS
✅ SVR done for LVS | Best C: 1 | Epsilon: 0.01

🟣 SVR - Stock 117/975: CAH
✅ SVR done for CAH | Best C: 1 | Epsilon: 0.1

🟣 SVR - Stock 118/975: WAB
✅ SVR done for WAB | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 1

✅ SVR done for COKE.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 211/975: TXRH.O
✅ SVR done for TXRH.O | Best C: 1 | Epsilon: 0.01

🟣 SVR - Stock 212/975: SWKS.O
✅ SVR done for SWKS.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 213/975: NBIX.O
✅ SVR done for NBIX.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 214/975: ITT
✅ SVR done for ITT | Best C: 10 | Epsilon: 0.01

🟣 SVR - Stock 215/975: VTRS.O
✅ SVR done for VTRS.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 216/975: SNX
✅ SVR done for SNX | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 217/975: AIZ
✅ SVR done for AIZ | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 218/975: RBC
✅ SVR done for RBC | Best C: 1 | Epsilon: 0.1

🟣 SVR - Stock 219/975: OVV
✅ SVR done for OVV | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 220/975: MANH.O
✅ SVR done for MANH.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 221/975: EMN
✅ SVR done for EMN | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 222/975: CCK
✅ SVR done for CCK | Best C: 1 | Epsilon: 0.01

🟣 S

✅ SVR done for NOV | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 314/975: RHI
✅ SVR done for RHI | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 315/975: SKY
✅ SVR done for SKY | Best C: 10 | Epsilon: 0.01

🟣 SVR - Stock 316/975: CRK
✅ SVR done for CRK | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 317/975: OPCH.O
✅ SVR done for OPCH.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 318/975: CGNX.O
✅ SVR done for CGNX.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 319/975: LSTR.O
✅ SVR done for LSTR.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 320/975: CRUS.O
✅ SVR done for CRUS.O | Best C: 1 | Epsilon: 0.01

🟣 SVR - Stock 321/975: CMC
✅ SVR done for CMC | Best C: 1 | Epsilon: 0.1

🟣 SVR - Stock 322/975: CYTK.O
✅ SVR done for CYTK.O | Best C: 10 | Epsilon: 0.01

🟣 SVR - Stock 323/975: QXO
✅ SVR done for QXO | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 324/975: IONS.O
✅ SVR done for IONS.O | Best C: 1 | Epsilon: 0.1

🟣 SVR - Stock 325/975: TKR
✅ SVR done for TKR | Best C: 10 | Epsilon: 0.1

🟣 SVR

✅ SVR done for PENN.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 417/975: JOE
✅ SVR done for JOE | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 418/975: HURN.O
✅ SVR done for HURN.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 419/975: JJSF.O
✅ SVR done for JJSF.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 420/975: GT.O
✅ SVR done for GT.O | Best C: 1 | Epsilon: 0.01

🟣 SVR - Stock 421/975: SAM
✅ SVR done for SAM | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 422/975: HP
✅ SVR done for HP | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 423/975: PSMT.O
✅ SVR done for PSMT.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 424/975: CPRX.O
✅ SVR done for CPRX.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 425/975: FORM.O
✅ SVR done for FORM.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 426/975: SYNA.O
✅ SVR done for SYNA.O | Best C: 1 | Epsilon: 0.01

🟣 SVR - Stock 427/975: QDEL.O
✅ SVR done for QDEL.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 428/975: VIAV.O
✅ SVR done for VIAV.O | Best C: 0.1 

✅ SVR done for IDT | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 519/975: BLFS.O
✅ SVR done for BLFS.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 520/975: GIII.O
✅ SVR done for GIII.O | Best C: 10 | Epsilon: 0.01

🟣 SVR - Stock 521/975: CRAI.O
✅ SVR done for CRAI.O | Best C: 10 | Epsilon: 0.01

🟣 SVR - Stock 522/975: IMKTA.O
✅ SVR done for IMKTA.O | Best C: 1 | Epsilon: 0.01

🟣 SVR - Stock 523/975: OPK.O
✅ SVR done for OPK.O | Best C: 10 | Epsilon: 0.1

🟣 SVR - Stock 524/975: SAFT.O
✅ SVR done for SAFT.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 525/975: RES
✅ SVR done for RES | Best C: 10 | Epsilon: 0.01

🟣 SVR - Stock 526/975: UCTT.O
✅ SVR done for UCTT.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 527/975: GYRE.O
✅ SVR done for GYRE.O | Best C: 10 | Epsilon: 0.01

🟣 SVR - Stock 528/975: MODG.K
✅ SVR done for MODG.K | Best C: 1 | Epsilon: 0.01

🟣 SVR - Stock 529/975: TILE.O
✅ SVR done for TILE.O | Best C: 1 | Epsilon: 0.1

🟣 SVR - Stock 530/975: MSEX.O
✅ SVR done for MSEX.O | Best C

✅ SVR done for TWI | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 620/975: HZO
✅ SVR done for HZO | Best C: 1 | Epsilon: 0.01

🟣 SVR - Stock 621/975: CLMB.O
✅ SVR done for CLMB.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 622/975: MLR
✅ SVR done for MLR | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 623/975: CCRN.O
✅ SVR done for CCRN.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 624/975: YORW.O
✅ SVR done for YORW.O | Best C: 1 | Epsilon: 0.01

🟣 SVR - Stock 625/975: BBW
✅ SVR done for BBW | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 626/975: VLGEA.O
✅ SVR done for VLGEA.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 627/975: LGTY.O
✅ SVR done for LGTY.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 628/975: PBT
✅ SVR done for PBT | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 629/975: MYE
✅ SVR done for MYE | Best C: 1 | Epsilon: 0.01

🟣 SVR - Stock 630/975: WNC
✅ SVR done for WNC | Best C: 1 | Epsilon: 0.1

🟣 SVR - Stock 631/975: GTN
✅ SVR done for GTN | Best C: 1 | Epsilon: 0.01

🟣 SVR - S

✅ SVR done for FXNC.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 722/975: SWKH.O
✅ SVR done for SWKH.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 723/975: UTMD.O
✅ SVR done for UTMD.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 724/975: LAKE.O
✅ SVR done for LAKE.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 725/975: GENC.K
✅ SVR done for GENC.K | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 726/975: LTBR.O
✅ SVR done for LTBR.O | Best C: 1 | Epsilon: 0.01

🟣 SVR - Stock 727/975: HQI.O
✅ SVR done for HQI.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 728/975: STRT.O
✅ SVR done for STRT.O | Best C: 10 | Epsilon: 0.1

🟣 SVR - Stock 729/975: RELL.O
✅ SVR done for RELL.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 730/975: EVC
✅ SVR done for EVC | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 731/975: FENC.O
✅ SVR done for FENC.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 732/975: SMID.O
✅ SVR done for SMID.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 733/975: CATX.K
✅ SVR done for CATX.K

✅ SVR done for INO.O | Best C: 10 | Epsilon: 0.1

🟣 SVR - Stock 822/975: IOR
✅ SVR done for IOR | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 823/975: MLSS.K
✅ SVR done for MLSS.K | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 824/975: AREN.K
✅ SVR done for AREN.K | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 825/975: VERU.O
✅ SVR done for VERU.O | Best C: 10 | Epsilon: 0.01

🟣 SVR - Stock 826/975: ASYS.O
✅ SVR done for ASYS.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 827/975: ARMP.K
✅ SVR done for ARMP.K | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 828/975: ASRT.O
✅ SVR done for ASRT.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 829/975: FKWL.O
✅ SVR done for FKWL.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 830/975: DOMH.O
✅ SVR done for DOMH.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 831/975: DLHC.O
✅ SVR done for DLHC.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 832/975: PED
✅ SVR done for PED | Best C: 1 | Epsilon: 0.01

🟣 SVR - Stock 833/975: ALTS.O
✅ SVR done for ALTS.O | 

✅ SVR done for TAIT.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 923/975: JCTC.O
✅ SVR done for JCTC.O | Best C: 1 | Epsilon: 0.1

🟣 SVR - Stock 924/975: CLRB.O
✅ SVR done for CLRB.O | Best C: 10 | Epsilon: 0.1

🟣 SVR - Stock 925/975: GOVX.O
✅ SVR done for GOVX.O | Best C: 10 | Epsilon: 0.01

🟣 SVR - Stock 926/975: GPUS.K
✅ SVR done for GPUS.K | Best C: 1 | Epsilon: 0.1

🟣 SVR - Stock 927/975: ERNA.O
✅ SVR done for ERNA.O | Best C: 10 | Epsilon: 0.01

🟣 SVR - Stock 928/975: AIRI.K
✅ SVR done for AIRI.K | Best C: 1 | Epsilon: 0.1

🟣 SVR - Stock 929/975: SMSI.O
✅ SVR done for SMSI.O | Best C: 0.1 | Epsilon: 0.01

🟣 SVR - Stock 930/975: STRM.O
✅ SVR done for STRM.O | Best C: 10 | Epsilon: 0.01

🟣 SVR - Stock 931/975: IMNN.O
✅ SVR done for IMNN.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 932/975: AWX
✅ SVR done for AWX | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 933/975: BCDA.O
✅ SVR done for BCDA.O | Best C: 0.1 | Epsilon: 0.1

🟣 SVR - Stock 934/975: MTR
✅ SVR done for MTR | Best C: 0

### Hyperparameter Tuning - DecisionTreeRegressor

In [14]:
# Define the Fama-French factor columns
ff_3_factors = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RF']

# Get list of stock columns (everything except the FF factors and 'Period')
stock_columns = [col for col in three_ff_cleaned_df.columns if col not in ff_3_factors]

# Start global timer
start_all = time.time()

# ------------------ Decision Tree Regression ------------------

all_dt_results = []
best_dt_models = {}

# Start global timer
start_all = time.time()

for idx, stock in enumerate(stock_columns):
    print(f"\n🌲 Decision Tree - Stock {idx + 1}/{len(stock_columns)}: {stock}")
    start_time = time.time()

    # Subset and process
    df_subset = three_ff_cleaned_df[ff_3_factors + [stock]].copy()
    df_subset['Stock_Return'] = df_subset[[stock]]
    df_subset['Excess_Mkt_Return'] = df_subset['Mkt-RF'] - df_subset['RF']
    df_subset = df_subset[['Date', 'Stock_Return', 'Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
    
    # Shift return behind by 1 to align predictors at t with return at t+1
    df_subset['Stock_Return_Shifted'] = df_subset['Stock_Return'].shift(-1)

    # Drop the first row which now has NaN in 'Stock_Return_Shifted'
    df_subset = df_subset.dropna()

    # Standardize the predictors
    scaler = StandardScaler()
    df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RF']] = scaler.fit_transform(df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RF']])

    dt_results = []

    for max_depth in tree_depths:
        model = DecisionTreeRegressor(max_depth=max_depth, random_state=random_seed)
        val_rmse_list = []

        for i in range(train_size, train_size + val_size - 3):
            train_end = month_periods[i - 1]
            val_start = month_periods[i]
            val_end = month_periods[i + 2]

            train_mask = df_subset['Date'].dt.to_period('M') <= train_end
            val_mask = (df_subset['Date'].dt.to_period('M') >= val_start) & \
                       (df_subset['Date'].dt.to_period('M') <= val_end)

            X_train = df_subset.loc[train_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
            y_train = df_subset.loc[train_mask, 'Stock_Return_Shifted']
            X_val = df_subset.loc[val_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
            y_val = df_subset.loc[val_mask, 'Stock_Return_Shifted']

            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            val_rmse_list.append(rmse)

        dt_results.append({
            'Stock': stock,
            'Max_Depth': max_depth,
            'Avg_Validation_RMSE': np.mean(val_rmse_list),
            'Model': model
        })

    dt_results.sort(key=lambda x: x['Avg_Validation_RMSE'])
    best_dt_models[stock] = dt_results[0]['Model']
    all_dt_results.append(dt_results[0])
    print(f"✅ Decision Tree done for {stock} | Best max_depth: {dt_results[0]['Max_Depth']}")
    
# End global timer
end_all = time.time()
print(f"\n🟩 Finished all stocks | Total Time: {end_all - start_all:.2f} seconds")


🌲 Decision Tree - Stock 1/975: AAPL.O
✅ Decision Tree done for AAPL.O | Best max_depth: 2

🌲 Decision Tree - Stock 2/975: NVDA.O
✅ Decision Tree done for NVDA.O | Best max_depth: 2

🌲 Decision Tree - Stock 3/975: MSFT.O
✅ Decision Tree done for MSFT.O | Best max_depth: 2

🌲 Decision Tree - Stock 4/975: AMZN.O
✅ Decision Tree done for AMZN.O | Best max_depth: 2

🌲 Decision Tree - Stock 5/975: LLY
✅ Decision Tree done for LLY | Best max_depth: 4

🌲 Decision Tree - Stock 6/975: WMT
✅ Decision Tree done for WMT | Best max_depth: 2

🌲 Decision Tree - Stock 7/975: XOM
✅ Decision Tree done for XOM | Best max_depth: 4

🌲 Decision Tree - Stock 8/975: MA
✅ Decision Tree done for MA | Best max_depth: 6

🌲 Decision Tree - Stock 9/975: UNH
✅ Decision Tree done for UNH | Best max_depth: 4

🌲 Decision Tree - Stock 10/975: ORCL.K
✅ Decision Tree done for ORCL.K | Best max_depth: 2

🌲 Decision Tree - Stock 11/975: COST.O
✅ Decision Tree done for COST.O | Best max_depth: 2

🌲 Decision Tree - Stock 12/9

✅ Decision Tree done for HES | Best max_depth: 2

🌲 Decision Tree - Stock 94/975: FICO.K
✅ Decision Tree done for FICO.K | Best max_depth: 8

🌲 Decision Tree - Stock 95/975: OXY
✅ Decision Tree done for OXY | Best max_depth: 2

🌲 Decision Tree - Stock 96/975: BKR.O
✅ Decision Tree done for BKR.O | Best max_depth: 2

🌲 Decision Tree - Stock 97/975: ROST.O
✅ Decision Tree done for ROST.O | Best max_depth: 2

🌲 Decision Tree - Stock 98/975: EW
✅ Decision Tree done for EW | Best max_depth: 4

🌲 Decision Tree - Stock 99/975: VLO
✅ Decision Tree done for VLO | Best max_depth: 2

🌲 Decision Tree - Stock 100/975: AME
✅ Decision Tree done for AME | Best max_depth: 2

🌲 Decision Tree - Stock 101/975: GLW
✅ Decision Tree done for GLW | Best max_depth: 2

🌲 Decision Tree - Stock 102/975: DHI
✅ Decision Tree done for DHI | Best max_depth: 2

🌲 Decision Tree - Stock 103/975: LHX
✅ Decision Tree done for LHX | Best max_depth: 2

🌲 Decision Tree - Stock 104/975: CTSH.O
✅ Decision Tree done for CTSH.O 

✅ Decision Tree done for GGG | Best max_depth: 4

🌲 Decision Tree - Stock 186/975: TER.O
✅ Decision Tree done for TER.O | Best max_depth: 2

🌲 Decision Tree - Stock 187/975: UNM
✅ Decision Tree done for UNM | Best max_depth: 2

🌲 Decision Tree - Stock 188/975: DLTR.O
✅ Decision Tree done for DLTR.O | Best max_depth: 2

🌲 Decision Tree - Stock 189/975: INSM.O
✅ Decision Tree done for INSM.O | Best max_depth: 2

🌲 Decision Tree - Stock 190/975: UTHR.O
✅ Decision Tree done for UTHR.O | Best max_depth: 2

🌲 Decision Tree - Stock 191/975: HOLX.O
✅ Decision Tree done for HOLX.O | Best max_depth: 2

🌲 Decision Tree - Stock 192/975: ILMN.O
✅ Decision Tree done for ILMN.O | Best max_depth: 2

🌲 Decision Tree - Stock 193/975: RVTY.K
✅ Decision Tree done for RVTY.K | Best max_depth: 2

🌲 Decision Tree - Stock 194/975: BMRN.O
✅ Decision Tree done for BMRN.O | Best max_depth: 2

🌲 Decision Tree - Stock 195/975: FLEX.O
✅ Decision Tree done for FLEX.O | Best max_depth: 2

🌲 Decision Tree - Stock 196/

✅ Decision Tree done for BRKR.O | Best max_depth: 2

🌲 Decision Tree - Stock 276/975: RLI
✅ Decision Tree done for RLI | Best max_depth: 2

🌲 Decision Tree - Stock 277/975: IVZ
✅ Decision Tree done for IVZ | Best max_depth: 4

🌲 Decision Tree - Stock 278/975: ONTO.K
✅ Decision Tree done for ONTO.K | Best max_depth: 2

🌲 Decision Tree - Stock 279/975: CWST.O
✅ Decision Tree done for CWST.O | Best max_depth: 2

🌲 Decision Tree - Stock 280/975: QRVO.O
✅ Decision Tree done for QRVO.O | Best max_depth: 2

🌲 Decision Tree - Stock 281/975: AAON.O
✅ Decision Tree done for AAON.O | Best max_depth: 4

🌲 Decision Tree - Stock 282/975: ALK
✅ Decision Tree done for ALK | Best max_depth: 6

🌲 Decision Tree - Stock 283/975: UFPI.O
✅ Decision Tree done for UFPI.O | Best max_depth: 2

🌲 Decision Tree - Stock 284/975: FLS
✅ Decision Tree done for FLS | Best max_depth: 2

🌲 Decision Tree - Stock 285/975: SNV
✅ Decision Tree done for SNV | Best max_depth: 2

🌲 Decision Tree - Stock 286/975: TGTX.O
✅ Decis

✅ Decision Tree done for DORM.O | Best max_depth: 2

🌲 Decision Tree - Stock 367/975: ALE
✅ Decision Tree done for ALE | Best max_depth: 2

🌲 Decision Tree - Stock 368/975: MMS
✅ Decision Tree done for MMS | Best max_depth: 4

🌲 Decision Tree - Stock 369/975: FIZZ.O
✅ Decision Tree done for FIZZ.O | Best max_depth: 2

🌲 Decision Tree - Stock 370/975: TDS
✅ Decision Tree done for TDS | Best max_depth: 2

🌲 Decision Tree - Stock 371/975: CBZ
✅ Decision Tree done for CBZ | Best max_depth: 2

🌲 Decision Tree - Stock 372/975: BCO
✅ Decision Tree done for BCO | Best max_depth: 6

🌲 Decision Tree - Stock 373/975: AVNT.K
✅ Decision Tree done for AVNT.K | Best max_depth: 2

🌲 Decision Tree - Stock 374/975: AVAV.O
✅ Decision Tree done for AVAV.O | Best max_depth: 2

🌲 Decision Tree - Stock 375/975: IESC.O
✅ Decision Tree done for IESC.O | Best max_depth: 2

🌲 Decision Tree - Stock 376/975: M
✅ Decision Tree done for M | Best max_depth: 2

🌲 Decision Tree - Stock 377/975: PRIM.K
✅ Decision Tree d

...

✅ Decision Tree done for AMSF.O | Best max_depth: 2

🌲 Decision Tree - Stock 546/975: KFRC.K
✅ Decision Tree done for KFRC.K | Best max_depth: 2

🌲 Decision Tree - Stock 547/975: BELFA.O
✅ Decision Tree done for BELFA.O | Best max_depth: 2

🌲 Decision Tree - Stock 548/975: THRM.O
✅ Decision Tree done for THRM.O | Best max_depth: 2

🌲 Decision Tree - Stock 549/975: LQDT.O
✅ Decision Tree done for LQDT.O | Best max_depth: 2

🌲 Decision Tree - Stock 550/975: MYGN.O
✅ Decision Tree done for MYGN.O | Best max_depth: 4

🌲 Decision Tree - Stock 551/975: NSSC.O
✅ Decision Tree done for NSSC.O | Best max_depth: 2

🌲 Decision Tree - Stock 552/975: UTL
✅ Decision Tree done for UTL | Best max_depth: 2

🌲 Decision Tree - Stock 553/975: WOLF.K
✅ Decision Tree done for WOLF.K | Best max_depth: 2

🌲 Decision Tree - Stock 554/975: KSS
✅ Decision Tree done for KSS | Best max_depth: 2

🌲 Decision Tree - Stock 555/975: CBRL.O
✅ Decision Tree done for CBRL.O | Best max_depth: 8

🌲 Decision Tree - Stock 556

✅ Decision Tree done for ALT.O | Best max_depth: 2

🌲 Decision Tree - Stock 635/975: EGY
✅ Decision Tree done for EGY | Best max_depth: 2

🌲 Decision Tree - Stock 636/975: EYPT.O
✅ Decision Tree done for EYPT.O | Best max_depth: 2

🌲 Decision Tree - Stock 637/975: SVRA.O
✅ Decision Tree done for SVRA.O | Best max_depth: 4

🌲 Decision Tree - Stock 638/975: TTI
✅ Decision Tree done for TTI | Best max_depth: 2

🌲 Decision Tree - Stock 639/975: CLFD.O
✅ Decision Tree done for CLFD.O | Best max_depth: 2

🌲 Decision Tree - Stock 640/975: NWPX.O
✅ Decision Tree done for NWPX.O | Best max_depth: 2

🌲 Decision Tree - Stock 641/975: ORKA.O
✅ Decision Tree done for ORKA.O | Best max_depth: 2

🌲 Decision Tree - Stock 642/975: DHIL.O
✅ Decision Tree done for DHIL.O | Best max_depth: 6

🌲 Decision Tree - Stock 643/975: MITK.O
✅ Decision Tree done for MITK.O | Best max_depth: 2

🌲 Decision Tree - Stock 644/975: SRDX.O
✅ Decision Tree done for SRDX.O | Best max_depth: 2

🌲 Decision Tree - Stock 645/97

✅ Decision Tree done for UTMD.O | Best max_depth: 2

🌲 Decision Tree - Stock 724/975: LAKE.O
✅ Decision Tree done for LAKE.O | Best max_depth: 2

🌲 Decision Tree - Stock 725/975: GENC.K
✅ Decision Tree done for GENC.K | Best max_depth: 2

🌲 Decision Tree - Stock 726/975: LTBR.O
✅ Decision Tree done for LTBR.O | Best max_depth: 2

🌲 Decision Tree - Stock 727/975: HQI.O
✅ Decision Tree done for HQI.O | Best max_depth: 2

🌲 Decision Tree - Stock 728/975: STRT.O
✅ Decision Tree done for STRT.O | Best max_depth: 2

🌲 Decision Tree - Stock 729/975: RELL.O
✅ Decision Tree done for RELL.O | Best max_depth: 2

🌲 Decision Tree - Stock 730/975: EVC
✅ Decision Tree done for EVC | Best max_depth: 2

🌲 Decision Tree - Stock 731/975: FENC.O
✅ Decision Tree done for FENC.O | Best max_depth: 2

🌲 Decision Tree - Stock 732/975: SMID.O
✅ Decision Tree done for SMID.O | Best max_depth: 2

🌲 Decision Tree - Stock 733/975: CATX.K
✅ Decision Tree done for CATX.K | Best max_depth: 2

🌲 Decision Tree - Stock 7

✅ Decision Tree done for HNNA.O | Best max_depth: 2

🌲 Decision Tree - Stock 812/975: RRGB.O
✅ Decision Tree done for RRGB.O | Best max_depth: 4

🌲 Decision Tree - Stock 813/975: UBCP.O
✅ Decision Tree done for UBCP.O | Best max_depth: 2

🌲 Decision Tree - Stock 814/975: MNOV.O
✅ Decision Tree done for MNOV.O | Best max_depth: 2

🌲 Decision Tree - Stock 815/975: INVE.O
✅ Decision Tree done for INVE.O | Best max_depth: 2

🌲 Decision Tree - Stock 816/975: SGA.O
✅ Decision Tree done for SGA.O | Best max_depth: 2

🌲 Decision Tree - Stock 817/975: ESP
✅ Decision Tree done for ESP | Best max_depth: 2

🌲 Decision Tree - Stock 818/975: OPTT.K
✅ Decision Tree done for OPTT.K | Best max_depth: 2

🌲 Decision Tree - Stock 819/975: CODA.O
✅ Decision Tree done for CODA.O | Best max_depth: 2

🌲 Decision Tree - Stock 820/975: AUBN.O
✅ Decision Tree done for AUBN.O | Best max_depth: 2

🌲 Decision Tree - Stock 821/975: INO.O
✅ Decision Tree done for INO.O | Best max_depth: 2

🌲 Decision Tree - Stock 822

✅ Decision Tree done for DAIO.O | Best max_depth: 4

🌲 Decision Tree - Stock 901/975: JOB
✅ Decision Tree done for JOB | Best max_depth: 2

🌲 Decision Tree - Stock 902/975: RVP
✅ Decision Tree done for RVP | Best max_depth: 4

🌲 Decision Tree - Stock 903/975: INTZ.O
✅ Decision Tree done for INTZ.O | Best max_depth: 2

🌲 Decision Tree - Stock 904/975: NAII.O
✅ Decision Tree done for NAII.O | Best max_depth: 2

🌲 Decision Tree - Stock 905/975: CVV.O
✅ Decision Tree done for CVV.O | Best max_depth: 2

🌲 Decision Tree - Stock 906/975: LSTA.O
✅ Decision Tree done for LSTA.O | Best max_depth: 2

🌲 Decision Tree - Stock 907/975: PTN
✅ Decision Tree done for PTN | Best max_depth: 2

🌲 Decision Tree - Stock 908/975: CREX.O
✅ Decision Tree done for CREX.O | Best max_depth: 2

🌲 Decision Tree - Stock 909/975: MXC
✅ Decision Tree done for MXC | Best max_depth: 4

🌲 Decision Tree - Stock 910/975: MTEX.O
✅ Decision Tree done for MTEX.O | Best max_depth: 4

🌲 Decision Tree - Stock 911/975: NNVC.K
✅ D

### Hyperparameter Tuning - XGBoost

In [15]:
# Define the Fama-French factor columns
ff_3_factors = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RF']

# Get list of stock columns (everything except the FF factors and 'Period')
stock_columns = [col for col in three_ff_cleaned_df.columns if col not in ff_3_factors]

# ------------------ XGBoost ------------------

all_xgb_results = []
best_xgb_models = {}

for idx, stock in enumerate(stock_columns):
    print(f"\n🟢 XGBoost - Stock {idx + 1}/{len(stock_columns)}: {stock}")

    # Subset and process
    df_subset = three_ff_cleaned_df[ff_3_factors + [stock]].copy()
    df_subset['Stock_Return'] = df_subset[[stock]]
    df_subset['Excess_Mkt_Return'] = df_subset['Mkt-RF'] - df_subset['RF']
    df_subset = df_subset[['Date', 'Stock_Return', 'Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
    
    # Shift return behind by 1 to align predictors at t with return at t+1
    df_subset['Stock_Return_Shifted'] = df_subset['Stock_Return'].shift(-1)

    # Drop the first row which now has NaN in 'Stock_Return_Shifted'
    df_subset = df_subset.dropna()

    # Standardize the predictors
    scaler = StandardScaler()
    df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RF']] = scaler.fit_transform(df_subset[['Excess_Mkt_Return', 'SMB', 'HML', 'RF']])

    xgb_results = []

    for n_estimators, lr in xgb_params:
        model = XGBRegressor(n_estimators=n_estimators, learning_rate=lr, random_state=random_seed)
        val_rmse_list = []

        for i in range(train_size, train_size + val_size - 3):
            train_end = month_periods[i - 1]
            val_start = month_periods[i]
            val_end = month_periods[i + 2]

            train_mask = df_subset['Date'].dt.to_period('M') <= train_end
            val_mask = (df_subset['Date'].dt.to_period('M') >= val_start) & (df_subset['Date'].dt.to_period('M') <= val_end)

            X_train = df_subset.loc[train_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
            y_train = df_subset.loc[train_mask, 'Stock_Return_Shifted']
            X_val = df_subset.loc[val_mask, ['Excess_Mkt_Return', 'SMB', 'HML', 'RF']]
            y_val = df_subset.loc[val_mask, 'Stock_Return_Shifted']

            model.fit(X_train, y_train)
            rmse = np.sqrt(mean_squared_error(y_val, model.predict(X_val)))
            val_rmse_list.append(rmse)

        xgb_results.append({
            'Stock': stock,
            'n_estimators': n_estimators,
            'learning_rate': lr,
            'Avg_Validation_RMSE': np.mean(val_rmse_list),
            'Model': model
        })

    xgb_results.sort(key=lambda x: x['Avg_Validation_RMSE'])
    best_xgb_models[stock] = xgb_results[0]['Model']
    all_xgb_results.append(xgb_results[0])
    print(f"✅ XGBoost done for {stock} | Estimators: {xgb_results[0]['n_estimators']} | LR: {xgb_results[0]['learning_rate']}")

# End global timer
end_all = time.time()
print(f"\n🟩 Finished all stocks | Total Time: {end_all - start_all:.2f} seconds")


🟢 XGBoost - Stock 1/975: AAPL.O
✅ XGBoost done for AAPL.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 2/975: NVDA.O
✅ XGBoost done for NVDA.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 3/975: MSFT.O
✅ XGBoost done for MSFT.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 4/975: AMZN.O
✅ XGBoost done for AMZN.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 5/975: LLY
✅ XGBoost done for LLY | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 6/975: WMT
✅ XGBoost done for WMT | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 7/975: XOM
✅ XGBoost done for XOM | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 8/975: MA
✅ XGBoost done for MA | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 9/975: UNH
✅ XGBoost done for UNH | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 10/975: ORCL.K
✅ XGBoost done for ORCL.K | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 11/975: COST.O
✅ XGBoost done for COST.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 12/975: NFLX.O
✅ XGBoost done for NFLX.O | 

✅ XGBoost done for ROST.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 98/975: EW
✅ XGBoost done for EW | Estimators: 50 | LR: 0.1

🟢 XGBoost - Stock 99/975: VLO
✅ XGBoost done for VLO | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 100/975: AME
✅ XGBoost done for AME | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 101/975: GLW
✅ XGBoost done for GLW | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 102/975: DHI
✅ XGBoost done for DHI | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 103/975: LHX
✅ XGBoost done for LHX | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 104/975: CTSH.O
✅ XGBoost done for CTSH.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 105/975: PWR
✅ XGBoost done for PWR | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 106/975: F
✅ XGBoost done for F | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 107/975: EA.O
✅ XGBoost done for EA.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 108/975: TTWO.O
✅ XGBoost done for TTWO.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stoc

✅ XGBoost done for ILMN.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 193/975: RVTY.K
✅ XGBoost done for RVTY.K | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 194/975: BMRN.O
✅ XGBoost done for BMRN.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 195/975: FLEX.O
✅ XGBoost done for FLEX.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 196/975: INCY.O
✅ XGBoost done for INCY.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 197/975: JKHY.O
✅ XGBoost done for JKHY.O | Estimators: 100 | LR: 0.05

🟢 XGBoost - Stock 198/975: MORN.O
✅ XGBoost done for MORN.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 199/975: ALGN.O
✅ XGBoost done for ALGN.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 200/975: EWBC.O
✅ XGBoost done for EWBC.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 201/975: AKAM.O
✅ XGBoost done for AKAM.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 202/975: FIX
✅ XGBoost done for FIX | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 203/975: JNPR.K
✅ XGBoost done f

✅ XGBoost done for SNV | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 286/975: TGTX.O
✅ XGBoost done for TGTX.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 287/975: TFX
✅ XGBoost done for TFX | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 288/975: CELH.O
✅ XGBoost done for CELH.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 289/975: EAT
✅ XGBoost done for EAT | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 290/975: PEGA.O
✅ XGBoost done for PEGA.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 291/975: OSK
✅ XGBoost done for OSK | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 292/975: LNC
✅ XGBoost done for LNC | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 293/975: AWI
✅ XGBoost done for AWI | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 294/975: CHH
✅ XGBoost done for CHH | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 295/975: WEX
✅ XGBoost done for WEX | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 296/975: CORT.O
✅ XGBoost done for CORT.O | Estimators: 200 | LR: 0.01


✅ XGBoost done for ICUI.O | Estimators: 50 | LR: 0.1

🟢 XGBoost - Stock 380/975: KFY
✅ XGBoost done for KFY | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 381/975: RDNT.O
✅ XGBoost done for RDNT.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 382/975: NWE.O
✅ XGBoost done for NWE.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 383/975: SM
✅ XGBoost done for SM | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 384/975: SMTC.O
✅ XGBoost done for SMTC.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 385/975: CCOI.O
✅ XGBoost done for CCOI.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 386/975: PLXS.O
✅ XGBoost done for PLXS.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 387/975: NSP
✅ XGBoost done for NSP | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 388/975: HAE
✅ XGBoost done for HAE | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 389/975: POWI.O
✅ XGBoost done for POWI.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 390/975: AVA
✅ XGBoost done for AVA | Estimators: 200 | 

...

✅ XGBoost done for CECO.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 565/975: ECPG.O
✅ XGBoost done for ECPG.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 566/975: PRA
✅ XGBoost done for PRA | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 567/975: AVXL.O
✅ XGBoost done for AVXL.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 568/975: COHU.O
✅ XGBoost done for COHU.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 569/975: AMSC.O
✅ XGBoost done for AMSC.O | Estimators: 100 | LR: 0.05

🟢 XGBoost - Stock 570/975: HCKT.O
✅ XGBoost done for HCKT.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 571/975: AIOT.O
✅ XGBoost done for AIOT.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 572/975: PRAA.O
✅ XGBoost done for PRAA.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 573/975: AXGN.O
✅ XGBoost done for AXGN.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 574/975: OSBC.O
✅ XGBoost done for OSBC.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 575/975: UFCS.O
✅ XGBoost done f

✅ XGBoost done for RIGL.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 657/975: TRAK.K
✅ XGBoost done for TRAK.K | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 658/975: HRTX.O
✅ XGBoost done for HRTX.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 659/975: DDD
✅ XGBoost done for DDD | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 660/975: APPS.O
✅ XGBoost done for APPS.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 661/975: MSB
✅ XGBoost done for MSB | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 662/975: PNRG.O
✅ XGBoost done for PNRG.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 663/975: ACTG.O
✅ XGBoost done for ACTG.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 664/975: HVT
✅ XGBoost done for HVT | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 665/975: ATNI.O
✅ XGBoost done for ATNI.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 666/975: MTW
✅ XGBoost done for MTW | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 667/975: MATV.K
✅ XGBoost done for MATV.K | Estima

✅ XGBoost done for CLAR.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 749/975: FEIM.O
✅ XGBoost done for FEIM.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 750/975: OPOF.O
✅ XGBoost done for OPOF.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 751/975: CCRD.K
✅ XGBoost done for CCRD.K | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 752/975: PHX
✅ XGBoost done for PHX | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 753/975: STRS.O
✅ XGBoost done for STRS.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 754/975: INSG.O
✅ XGBoost done for INSG.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 755/975: FLL.O
✅ XGBoost done for FLL.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 756/975: OPRX.O
✅ XGBoost done for OPRX.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 757/975: SRI
✅ XGBoost done for SRI | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 758/975: KEQU.O
✅ XGBoost done for KEQU.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 759/975: SAVA.O
✅ XGBoost done for SAVA.

✅ XGBoost done for WFCF.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 840/975: LINK.O
✅ XGBoost done for LINK.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 841/975: CTSO.O
✅ XGBoost done for CTSO.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 842/975: CNVS.O
✅ XGBoost done for CNVS.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 843/975: CNTY.O
✅ XGBoost done for CNTY.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 844/975: MIND.O
✅ XGBoost done for MIND.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 845/975: SOTK.O
✅ XGBoost done for SOTK.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 846/975: RMTI.O
✅ XGBoost done for RMTI.O | Estimators: 100 | LR: 0.05

🟢 XGBoost - Stock 847/975: CVGI.O
✅ XGBoost done for CVGI.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 848/975: MBOT.O
✅ XGBoost done for MBOT.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 849/975: LEE.O
✅ XGBoost done for LEE.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 850/975: CMTL.O
✅ XGBoost do

✅ XGBoost done for IMNN.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 932/975: AWX
✅ XGBoost done for AWX | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 933/975: BCDA.O
✅ XGBoost done for BCDA.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 934/975: MTR
✅ XGBoost done for MTR | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 935/975: PRPH.O
✅ XGBoost done for PRPH.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 936/975: SSKN.O
✅ XGBoost done for SSKN.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 937/975: ASTC.O
✅ XGBoost done for ASTC.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 938/975: PMCB.O
✅ XGBoost done for PMCB.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 939/975: PRPO.O
✅ XGBoost done for PRPO.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 940/975: BBGI.O
✅ XGBoost done for BBGI.O | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 941/975: MSN
✅ XGBoost done for MSN | Estimators: 200 | LR: 0.01

🟢 XGBoost - Stock 942/975: BCLI.O
✅ XGBoost done for BCLI.O | 

### Output the best tuned config for each model for each stock

In [16]:
# Step 1: Build rows from each model's best result list
rows = []

for stock in stock_columns:
    row = {'Stock': stock}

    # Ridge
    ridge = next((r for r in all_ridge_results if r['Stock'] == stock), None)
    if ridge:
        row['Ridge'] = f"α={ridge['Alpha']}"

    # Lasso
    lasso = next((r for r in all_lasso_results if r['Stock'] == stock), None)
    if lasso:
        row['Lasso'] = f"α={lasso['Alpha']}"

    # ElasticNet
    elastic = next((r for r in all_elastic_results if r['Stock'] == stock), None)
    if elastic:
        row['ElasticNet'] = f"α={elastic['Alpha']}, l1={elastic['L1_Ratio']}"

    # SVR
    svr = next((r for r in all_svr_results if r['Stock'] == stock), None)
    if svr:
        row['SVR'] = f"C={svr['C']}, ε={svr['Epsilon']}"

    # Decision Tree
    dt = next((r for r in all_dt_results if r['Stock'] == stock), None)
    if dt:
        row['DecisionTree'] = f"depth={dt['Max_Depth']}"
        
    # XGBoost
    xgb = next((r for r in all_xgb_results if r['Stock'] == stock), None)
    if xgb:
        row['XGBoost'] = f"est={xgb['n_estimators']}, lr={xgb['learning_rate']}"

    rows.append(row)

# Step 2: Create DataFrame
model_summary_df = pd.DataFrame(rows)
# Define output path
excel_path = 'output/models/3ff_tuned_para.xlsx'

# Export to Excel
model_summary_df.to_excel(excel_path, index=False)